# (F) summer change detection and classification
**REQUIREMENTS : (A), (C) and (E)**

This program performs the summer change detection and classification based on the S2 data available for the summer months which are defined as the months between May and September.

This programm is therefore part of the summer process only.

It is meant to be call once a year (after Octobre).

The following function is used to perform this task:

> **summer_change_classification(sites, annee, dteExe)**

This function performed the following steps:

**1) Creation/Determination of the result table**

This table is names **classification_chg_summer**

**2) For each site, tables containing smoothed profiles of S2 indexes are fetched**

NDVI, BAI, BI, BI2, SBI smoothed profile tables have been generated by (C).

COMPO smoothed profile tables have been generated by (E).

**3) For each site, computation of index summer amplitude (variation between current and previous summer)**

Summer months are considered to be between May and September.

Smoothed index values are averaged over the current summer and over the previous summer.

For a given index, its amplitude is the difference its current summer average and its previous summer average.

**4) For each site, caracterisation of changes in vegetation, soil and building based on index amplitudes**

Vegetation change caracterisation is based on NDVI analysis.
Soil change caracterisation is based on BAI analysis.
Building change caracterisation is based on BI, BI2 and SBI analysis.

**Code interpretation key**
> 0 No change

> 1 Change (no additional information)

> 2 Increase

> 3 Decrease

> 4 Possible change (no additional information)

> 5 Possible increase

> 6 Possible decrease

## Import libraries and functions

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

## Definition of the summer_change classification function

In [ ]:
def summer_change_classification(sites, annee, dteExe):
    """ This function performs the summer change detection and classification based on the S2 data available for the summer months 
        which are defined as the months between May and September. 
        Results of this function are returned in the following table: "classification_chg_summer".
            
            Parameters:
                sites (list): list of sites (sar_id_segment) to process
                annee (interger): year of the summer to process (YYYY)
                dteExe (date): execution date (YYYYMMDD)
        """
    ## CONNECTION DU DB
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')
    
    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Open connexion to DB
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None

    ## STARTING SUMMER CHANGE CLASSIFICATION 

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

    ## (RE-)CREATION OF THE RESULT

        table_name = 'classification_chg_summer'

        # strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)
        # cur.execute(strSQL)
        # conn.commit()

        strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (ID_Segment TEXT NOT NULL, dteExe DATE, periode TEXT, NDVI_amplitude NUMERIC(4,3), BAI_amplitude NUMERIC, BI_amplitude NUMERIC, BI2_amplitude NUMERIC, SBI_amplitude NUMERIC, COMPO_amplitude NUMERIC, vegetation INTEGER, building INTEGER, soil INTEGER, UNIQUE(ID_Segment, periode));'.format(table_name)
        cur.execute(strSQL)

        conn.commit()

#         # Liste des sites pour lesquels on a des mesures dans l'intervalle donné (de mai première année à août dernière année)
#         cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats WHERE acq_date BETWEEN \'{0}-05-01\' AND \'{1}-08-31\' ORDER BY sar_id_segment;'.format(annee-1, annee))
#         SARs = cur.fetchall()
        SARs = sites
    
     ## INDEX AMPLITUDE CALCULATION
        
        # For each site and index concerned and each year of our interval (annee and annee-1), note the differences in summer mean)
        for SAR in SARs:
            site = SAR['sar_id_segment']

            for indice in ['NDVI', 'BAI', 'BI', 'BI2', 'SBI', 'COMPO']:
                
                # Checking existence of tables containing the smoothed indexes (one table per index per site)
                
                table_smooth = '{0}_{1}_{2}_smoothed'.format(site, indice, dteExe)
                strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                cur.execute(strSQL)

                # If table_smooth exists, index summer means are computed for annee and annee-1
                
                INDICE_EXISTS = cur.fetchone()[0]

                if INDICE_EXISTS == True:                           
                    strSQL = '''SELECT CONCAT(a.annee, \'-\', p.annee) AS periode, a.moyenne, p.moyenne AS moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg 
                                FROM (SELECT EXTRACT(YEAR FROM dte) AS annee, AVG(indice) AS moyenne FROM "{0}" WHERE EXTRACT(MONTH FROM dte) BETWEEN 5 AND 8 GROUP BY annee) a, 
                                (SELECT EXTRACT(YEAR FROM dte) AS annee, AVG(indice) AS moyenne FROM "{0}" WHERE EXTRACT(MONTH FROM dte) BETWEEN 5 AND 8 GROUP BY annee) p 
                                WHERE p.annee = a.annee - 1 AND a.annee = {1};'''.format(table_smooth, annee)
                    cur2.execute(strSQL)            
                    lignes = cur2.fetchall()
                    
                    # Amplidude (difference between two summers) is computed and save in the result table
                    
                    for ligne in lignes:
                        periode = ligne['periode']
                        moyenne_chg = ligne['moyenne_chg']

                        strSQL = '''INSERT INTO "{0}" (ID_Segment, dteExe, periode, {1}_amplitude) VALUES (\'{2}\', \'{3}\', \'{4}\', \'{5}\') 
                                    ON CONFLICT (ID_Segment, periode) DO UPDATE SET {1}_amplitude = \'{5}\', dteExe = \'{3}\';'''.format(table_name, indice, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', periode, moyenne_chg)
                        cur2.execute(strSQL)

            # Modification are saved
            conn.commit()
            
    ## CARACTERIZATION OF SUMMER CHANGE

            # Comparing the amplitude of change in the indices to thresholds to define the type of change
            strSQL = 'SELECT * FROM "{0}" WHERE ID_Segment = \'{1}\' AND dteExe = \'{2}\';'.format(table_name, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}')
            cur.execute(strSQL)
            lignes = cur.fetchall()

            for ligne in lignes:
                periode = ligne['periode']
                NDVI_amplitude = ligne['ndvi_amplitude']
                BAI_amplitude = ligne['bai_amplitude']
                BI_amplitude = ligne['bi_amplitude']
                BI2_amplitude = ligne['bi2_amplitude']
                SBI_amplitude = ligne['sbi_amplitude']

                # Vegetation change
                if NDVI_amplitude != None:
                    if NDVI_amplitude >= 0.2:
                        vegetation = 2  # « Increase in vegetation »

                    elif NDVI_amplitude >= 0.1:
                        vegetation = 5  # « Possible increase in vegetation »

                    elif NDVI_amplitude <= -0.2:
                        vegetation = 3  # « Decrease in vegetation »

                    elif NDVI_amplitude <= -0.1:
                        vegetation = 6  # « Possible decrease in vegetation »

                    else:
                        vegetation = 0  # « No change in vegetation »

                # Soil change
                if BAI_amplitude != None:

                    if abs(BAI_amplitude) >= 0.1:
                        soil = 1  # « Change in soil »

                    elif abs(BAI_amplitude) >= 0.05:
                        soil = 4  # « Possible change in soil »

                    else:
                        soil = 0  # « No change in soil »

                # Building change
                if None not in [BI_amplitude, BI2_amplitude, SBI_amplitude]:

                    if abs(BI_amplitude) >= 250 or abs(BI2_amplitude) >= 250 or abs(SBI_amplitude) >= 400:
                        building = 1  # « Change in building »

                    elif abs(BI_amplitude) >= 150 or abs(BI2_amplitude) >= 150 or abs(SBI_amplitude) >= 250:
                        building = 4  # « Possible change in building »

                    else :
                        building = 0      # «No change in building»

    ## SAVING RESULTS
    
                strSQL = 'UPDATE "{0}" SET vegetation = {1}, soil = {2}, building = {3} WHERE ID_Segment = \'{4}\' AND dteExe = \'{5}\' AND periode = \'{6}\';'.format(table_name, vegetation, soil, building, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', periode)
                cur2.execute(strSQL)

                # Posting all modifications
                conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if cur2 is not None:
            cur2.close()

        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

## Calling functions

In [ ]:
## Call config
dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']
dteExe = dates['exe']
summer = int(dates['summer'])

## Call sites_to_process
lstSARs = sites_to_process(dteDebut, dteFin)
 
## Call summer_change_classification
summer_change_classification(lstSARs, summer, dteExe)